In [1]:
import os
os.environ['MKL_NUM_THREADS'] = '1'
import numpy as np
import math

In [2]:
def read_graph(file):
    with open(file, 'r') as f:
        n, k = f.readline().split()
        n, k = int(n), int(k)

        grafo = np.zeros((n, n))

        for i, line in enumerate(f):
            grafo[i, [int(j) - 1 for j in line.split()]] = 1

    return grafo

In [3]:
dir='./BIPART/G124.16'
grafo1=read_graph(dir)
print(grafo1)

FileNotFoundError: [Errno 2] No such file or directory: './BIPART/G124.16'

In [3]:
test = np.array([[0,0,1,1,0,0],
                 [0,0,1,1,0,1],
                 [1,1,0,0,1,0],
                 [1,1,0,0,1,0],
                 [0,0,1,1,0,0],
                 [0,1,0,0,0,0]])

testp = np.array([1,1,1,0,0,0])

In [4]:
def cost(graph, p):
    pos_mat = np.zeros_like(graph)
    pos_mat[p==1,] = 1-p
    pos_mat[p==0,] = p

    return np.sum(graph * pos_mat)

In [5]:
def localsearch(graph, p_, n):
    cost_v = cost(graph, p_)
    recall = cost_v
    better_cost = recall

    p = np.copy(p_)
    new_p = np.copy(p_)
    better_sols = np.copy(p_)

    for _ in range(n):
        # go throug all possible changes in order without
        # computing the solutions and/or costs from scratch
        # self-loops are not allowed

        for i in np.nonzero(p==1)[0]:
            sub_cause_i = np.dot(graph[i,:],(1-p)) + np.dot(graph[:,i],(1-p))

            new_p[i] ^= 1

            for j in np.nonzero(p==0)[0]:

                new_p[j] ^= 1

                cost_v -= np.dot(graph[j,:],p) + np.dot(graph[:,j],p) + sub_cause_i

                # substracting graph[i, j] twice (also graph[i, i] and [j, j])

                cost_v += np.dot(graph[j,:],(1-new_p)) + np.dot(graph[:,j],(1-new_p)) + np.dot(graph[i,:],new_p) +np.dot(graph[:,i],new_p)

                # summing graph[i, j] twice

                if cost_v < better_cost:
                    better_sols = new_p.copy()
                    better_cost = cost_v

                cost_v = recall
                new_p[j] ^= 1

            new_p[i] ^= 1

        p = better_sols.copy()
        new_p = p.copy()
        recall = better_cost
        cost_v = recall

    return (better_cost, better_sols)


In [6]:
localsearch(test, testp, 1)

(6, array([0, 1, 1, 0, 0, 1]))

In [7]:
import time

n=1000
a=np.random.randint(0, 2, (n, n))
b=np.random.randint(0, 2, n)

In [8]:
d=time.time()
h=localsearch(a, b, 1)
print(time.time()-d)

5.478344202041626


In [10]:
file = open("save_test.txt", "w")

file.write(str(n) + " " + str(a.sum()) + "\n")

for i in range(n):
    write_to_file = " ".join([str(h + 1) for h in np.nonzero(a[i,:]==1)[0]])
    file.write(write_to_file)
    if i!=n-1:
        file.write("\n")

file.close()

In [101]:
def aco(graph, n, n_ants, alpha, beta, p):
    n_vertices = graph.shape[0]

    # initialize partition
    p_initial = np.zeros(n_vertices, dtype=int)
    p_initial[np.random.choice(np.array([i for i in range(n_vertices)]), size=n_vertices//2, replace=False)] = 1

    # compute cost of this initial partition
    cost_best = cost(graph, p_initial)
    p_best = p_initial.copy()

    # initialize pheromones and heuristic function
    nu = np.zeros(n_vertices)
    tau = np.zeros((n_vertices, 2))

    for i in range(graph.shape[0]):
        nu[i] = graph[i,:].sum()

        tau[i, p_initial[i]] = (np.dot(graph[i,:], 1 - p_initial) + 1) / (np.dot(graph[i,:], p_initial) + 1)
        tau[i, 1 - p_initial[i]] = 1/tau[i,p_initial[i]]

    # start loop
    max_vertices_in_0 = n_vertices//2
    max_vertices_in_1 = n_vertices - max_vertices_in_0

    for _ in range(n):

        # initialize matrix of partitions for each ant and costs vector for each ant
        p_ants = np.zeros((n_ants, n_vertices), dtype=int)
        c_ants = np.zeros(n_ants)

        for ant in range(n_ants):

            # initialize available vertices and set counter of vertices in each partition
            available_vertices = [j for j in range(n_vertices)]
            vertices_in_0 = 0
            vertices_in_1 = 0

            for i in range(n_vertices):
                # select partition to put edge
                s = np.random.choice(2,p=((max_vertices_in_0 - vertices_in_0) / (n_vertices - i), (max_vertices_in_1 - vertices_in_1) / (n_vertices - i)))
                if s:
                    vertices_in_1 += 1
                else:
                    vertices_in_0 += 1
                
                # select edge to put in partition
                probabilities = np.array([tau[v, s]**alpha * nu[v]**beta for v in available_vertices])
                probabilities /= probabilities.sum()
                v_choice = available_vertices.pop(np.random.choice(len(available_vertices), p=probabilities))
                
                # update the partition
                p_ants[ant, v_choice] = s

            # compute cost of partition
            c_ants[ant] = cost(graph, p_ants[ant,:])
        
        # select the best ant
        min_ant = np.argmin(c_ants)

        # update best cost and partitions
        if c_ants[min_ant] < cost_best:
            cost_best = c_ants[min_ant]
            p_best = p_ants[min_ant, :]

        # update pheromones with p_ants[min_ant, :] (the best from the iteration)
        dtau = np.zeros((n_vertices, 2))
        for i in range(graph.shape[0]):
            dtau[i, p_ants[min_ant, i]] = (np.dot(graph[i,:], 1 - p_ants[min_ant, :]) + 1) / (np.dot(graph[i,:], p_ants[min_ant, :]) + 1)
            dtau[i, 1 - p_ants[min_ant, i]] = 1/dtau[i,p_ants[min_ant, i]]

            tau = (1-p) * tau + p*dtau
    
    return cost_best, p_best

In [115]:
aco(a, 1, 1, 1, 1, 0.9)[1].sum()

500

In [114]:
aco(test, 5, 5, 1, 1, 0.9)

(6.0, array([0, 1, 1, 0, 0, 1]))